# Lab work #4
# Lypnytska Inna
# CS32
# The task contains 2 parts:
1. Play with [Tensorflow Playground](https://playground.tensorflow.org) and save a bunch of screenshots (at least 10).
2. Solve the classification problem on your dataset from the Lab #2: play with number of neurons, function activation and learning rate (and if you wish play with learning rate schedule)

In [6]:
import pandas as pd

# Завантажуємо датасет
df = pd.read_csv("CrabAgePrediction.csv")

# Видаляємо пропуски, якщо є
df.dropna(inplace=True)

# One-hot encoding для статі
df = pd.get_dummies(df, columns=["Sex"], drop_first=True)


In [7]:
q1 = df["Age"].quantile(0.25)
q3 = df["Age"].quantile(0.75)

def classify_age(age):
    if age <= q1:
        return 0  # Молодий
    elif age <= q3:
        return 1  # Дорослий
    else:
        return 2  # Старий

df["AgeClass"] = df["Age"].apply(classify_age)
df = df.drop("Age", axis=1)

# Перевірка кількості зразків у кожному класі
print(df["AgeClass"].value_counts())


AgeClass
1    1697
0    1293
2     903
Name: count, dtype: int64


In [8]:
from sklearn.model_selection import train_test_split

X = df.drop("AgeClass", axis=1)
y = df["AgeClass"]

# Стратифікований поділ (щоб пропорції класів збереглися)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")


Train size: 3114, Test size: 779


In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [10]:
import tensorflow as tf

y_train_ohe = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes=3)


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

def build_model(num_neurons=16, activation='relu', learning_rate=0.001):
    model = Sequential([
        Dense(num_neurons, activation=activation, input_shape=(X_train_scaled.shape[1],)),
        Dense(num_neurons, activation=activation),
        Dense(3, activation='softmax')  # 3 класи
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [12]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

params_list = [
    {'num_neurons': 16, 'activation': 'relu', 'learning_rate': 0.001},
    {'num_neurons': 32, 'activation': 'relu', 'learning_rate': 0.001},
    {'num_neurons': 16, 'activation': 'tanh', 'learning_rate': 0.001},
    {'num_neurons': 16, 'activation': 'sigmoid', 'learning_rate': 0.001},
    {'num_neurons': 16, 'activation': 'relu', 'learning_rate': 0.01},
]

for i, params in enumerate(params_list, 1):
    print(f"\n--- Експеримент {i}: {params} ---")
    model = build_model(**params)

    history = model.fit(
        X_train_scaled, y_train_ohe,
        validation_split=0.2,
        epochs=50,
        batch_size=16,
        verbose=0
    )

    test_loss, test_acc = model.evaluate(X_test_scaled, y_test_ohe, verbose=0)
    print(f"Точність на тесті: {test_acc:.4f}")

    y_pred_prob = model.predict(X_test_scaled)
    y_pred = np.argmax(y_pred_prob, axis=1)

    print("Classification Report:")
    print(classification_report(y_test, y_pred, digits=4))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))



--- Експеримент 1: {'num_neurons': 16, 'activation': 'relu', 'learning_rate': 0.001} ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Точність на тесті: 0.6765
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Classification Report:
              precision    recall  f1-score   support

           0     0.7080    0.7490    0.7280       259
           1     0.6374    0.6844    0.6600       339
           2     0.7163    0.5580    0.6273       181

    accuracy                         0.6765       779
   macro avg     0.6872    0.6638    0.6718       779
weighted avg     0.6792    0.6765    0.6750       779

Confusion Matrix:
[[194  61   4]
 [ 71 232  36]
 [  9  71 101]]

--- Експеримент 2: {'num_neurons': 32, 'activation': 'relu', 'learning_rate': 0.001} ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Точність на тесті: 0.6816
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Classification Report:
              precision    recall  f1-score   support

           0     0.7572    0.7104    0.7331       259
           1     0.6295    0.7168    0.6703       339
           2     0.6933    0.5746    0.6284       181

    accuracy                         0.6816       779
   macro avg     0.6934    0.6673    0.6773       779
weighted avg     0.6868    0.6816    0.6815       779

Confusion Matrix:
[[184  70   5]
 [ 55 243  41]
 [  4  73 104]]

--- Експеримент 3: {'num_neurons': 16, 'activation': 'tanh', 'learning_rate': 0.001} ---


...

Точність на тесті: 0.6919
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Classification Report:
              precision    recall  f1-score   support

           0     0.7788    0.6795    0.7258       259
           1     0.6321    0.7552    0.6882       339
           2     0.7230    0.5912    0.6505       181

    accuracy                         0.6919       779
   macro avg     0.7113    0.6753    0.6881       779
weighted avg     0.7020    0.6919    0.6919       779

Confusion Matrix:
[[176  79   4]
 [ 46 256  37]
 [  4  70 107]]

--- Експеримент 4: {'num_neurons': 16, 'activation': 'sigmoid', 'learning_rate': 0.001} ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Точність на тесті: 0.6881
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Classification Report:
              precision    recall  f1-score   support

           0     0.7375    0.7375    0.7375       259
           1     0.6499    0.7227    0.6844       339
           2     0.6993    0.5525    0.6173       181

    accuracy                         0.6881       779
   macro avg     0.6955    0.6709    0.6797       779
weighted avg     0.6905    0.6881    0.6864       779

Confusion Matrix:
[[191  60   8]
 [ 59 245  35]
 [  9  72 100]]

--- Експеримент 5: {'num_neurons': 16, 'activation': 'relu', 'learning_rate': 0.01} ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Точність на тесті: 0.6855
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Classification Report:
              precision    recall  f1-score   support

           0     0.7571    0.7220    0.7391       259
           1     0.6229    0.7552    0.6827       339
           2     0.7521    0.5028    0.6026       181

    accuracy                         0.6855       779
   macro avg     0.7107    0.6600    0.6748       779
weighted avg     0.6975    0.6855    0.6828       779

Confusion Matrix:
[[187  72   0]
 [ 53 256  30]
 [  7  83  91]]
